# Neural Net From Scratch

## Imports

In [51]:
import numpy as np
from random import random, seed

from sklearn.metrics import r2_score, accuracy_score

seed(0)

## Model

### Activation functions

In [2]:
class Linear():
    @staticmethod
    def f(x):
        return x
    @staticmethod
    def df(x):
        return x

class Relu():
    @staticmethod
    def f(x):
        return x * (x > 0)
    @staticmethod
    def df(x):
        return 1 * (x > 0)

class Sigmoid():
    @staticmethod
    def f(x):
        return 1.0 / (1.0 + np.exp(-x))
    @staticmethod
    def df(x):
        return Sigmoid.f(x) * (1 - Sigmoid.f(x))

### Loss functions

In [3]:
class MSE():
  @staticmethod
  def f(target, output):
    return np.mean((target - output) **2)
  
  @staticmethod 
  def df(target, output):
    return 2 * (target - output) / np.size(output)

class Crossentropy():
  @staticmethod
  def f(target, output):
    return np.mean(-target * np.log(output) - (1 - target) * np.log(1 - output))
  
  @staticmethod 
  def df(target, output):
    return ((1 -  target) / (1 - output) - target/output) / np.size(target)  

### Dense layer

In [4]:
class Layer:
    """Dit is een dense layer. Alle neuronen zijn fully connected"""
    def __init__(self, neurons, activation=Linear):
        self.neurons = neurons
        self.activation = activation
        self.weights = None
        self.biases = None
        self.lr = 0.02 # Snelheid waarmee de model traint -> staat nu hardcoded

    def initialize(self, prev_neurons):
        self.weights = np.random.normal(0.0, 1.0, (self.neurons, prev_neurons))
        self.biases = np.random.normal(size=(self.neurons, 1))

    def feed_forward(self, input_array) -> np.ndarray:
        # Input en output opslaan -> zijn nodig voor gradient descent
        self.inputs = input_array 

        # Y = W * I + B (matrix * vector)
        self.output = self.activation.f(np.dot(self.weights, self.inputs) + self.biases) 
        return self.output

    def gradient_descent(self, errors):
        """De waardes voor de minimum worden hier berekend 
        en worden de weights en biases geupdate"""
        
        delta_weights = self.lr * np.dot((errors * self.activation.df(self.output)), self.inputs.T)
        delta_biases = self.lr * errors * self.activation.df(self.output)

        self.weights = np.add(self.weights, delta_weights)
        self.biases = np.add(self.biases, delta_biases)

### Neurale netwerk

In [5]:
class Sequential():
    def __init__(self):
        self.layers = []

    def add(self, layer):
        # Initialiseer de laag met weights en biases alleen als het niet de input laag is.
        if len(self.layers) > 0:
            prev_neurons = self.layers[-1].neurons
            layer.initialize(prev_neurons)
        
        self.layers.append(layer)
        

    def predict(self, input_array: np.ndarray, transposed = False):
        assert isinstance(input_array, np.ndarray)
        
        output = input_array
        
        if not transposed:
            output = output.T

        for lay in self.layers[1:]:
            output = lay.feed_forward(output)
        return output

    def fit(self, input_array: np.ndarray ,output_array: np.ndarray, loss_function=MSE, epochs=10):
        assert isinstance(input_array, np.ndarray)
        assert isinstance(output_array, np.ndarray)

        for epoch in range(epochs):
            sum_errors = 0

            for x, y in zip(input_array, output_array):
                
                # feedforward
                prediction = self.predict(x.reshape(-1,1), transposed = True) # Reshape maakt ndmin 2 en transposed het al

                # houdt errors bij om het te visualiseren
                sum_errors += loss_function.f(y, prediction) 

                # backprop
                error = np.array(loss_function.df(y, prediction), ndmin=2) # gebruik afgeleide om weten of we de gewichten moeten verhogen of verlagen

                for lay in reversed(self.layers[1:]): # We moeten de eerste layer niet hebben. Dat is de input array en bevat eigenlijk geen weights of biases
                    lay.gradient_descent(error) # update
                    error = np.dot(lay.weights.T, error) # bereken error voor de nieuwe layer           
            sum_errors /= len(input_array)

            print(f"Epoch {epoch+1}/{epochs}:")
            print(f"Error: {sum_errors}\n")
        print(f"\nFinished Training\n{'=' * 50}")



## Testing

#### Regressie

Getallen optellen van 3

e.g. array van [0.23, 0.56, 0.14] = 0.93

In [52]:
X_train = np.array([[random() for _ in range(3)] for _ in range(1000)])
y_train = np.array([i[0] + i[1] + i[2] for i in X_train])

X_test = np.array([[3.1, 8.4, 4.0], [50.0, 32.3, 6.6],[0.4, 0.4, 0.12], [0.27, 0.32, 0.02], [0.1, 0.13, 0.32], [0.76, 0.22, 0], [0.26, 0.35, 0.05]]) 
y_test = np.array([[15.5, 88.9, 0.92, 0.61, 0.56, 0.98, 0.66]])

In [53]:
model = Sequential()

model.add(Layer(3, activation=Relu))
model.add(Layer(5, activation=Relu))
model.add(Layer(1, activation=Relu))

In [8]:
model.fit(X_train, y_train, epochs=10, loss_function=MSE)

Epoch 1/10:
Error: 0.1322190836082

Epoch 2/10:
Error: 0.0038251452373790434

Epoch 3/10:
Error: 0.0008561228017067467

Epoch 4/10:
Error: 0.00032669935647834344

Epoch 5/10:
Error: 0.00015867255169890314

Epoch 6/10:
Error: 9.433523050666762e-05

Epoch 7/10:
Error: 6.40051023775172e-05

Epoch 8/10:
Error: 4.751967527797353e-05

Epoch 9/10:
Error: 3.732336239759605e-05

Epoch 10/10:
Error: 3.097502077905794e-05


Finished Training


In [9]:
model.predict(np.array([[0.53, 0.11, 0.23]]))

array([[0.87064471]])

In [30]:
pred = model.predict(X_test)
pred = np.around(pred, 2)
pred

array([[15.43, 95.73,  0.92,  0.61,  0.55,  0.98,  0.66]])

In [34]:
r2_score(y_test.flatten(), pred.flatten())

0.9927959755813627

### Binary classification

In [59]:
def get_model() -> Sequential:
    model = Sequential()

    model.add(Layer(2, activation=Relu))
    model.add(Layer(5, activation=Relu))
    model.add(Layer(3, activation=Relu))
    model.add(Layer(1, activation=Sigmoid))

    return model


#### AND

In [58]:
X_train = np.array([[0,0], [0,1], [1,0], [1,1]])
y_train = np.array([0,0,0,1])

X_test = np.array([[1,0], [1,1], [0, 0]])
y_test = np.array([0,1,0])

In [60]:
model = get_model()

model.fit(X_train, y_train, epochs=1000)

Epoch 1/1000:
Error: 0.24874228621261033

Epoch 2/1000:
Error: 0.24218858484745862

Epoch 3/1000:
Error: 0.2325507978325289

Epoch 4/1000:
Error: 0.22030181582480776

Epoch 5/1000:
Error: 0.20528401772404065

Epoch 6/1000:
Error: 0.18877823991297848

Epoch 7/1000:
Error: 0.17204626649405555

Epoch 8/1000:
Error: 0.1560172035507125

Epoch 9/1000:
Error: 0.141222241262649

Epoch 10/1000:
Error: 0.12590215471249216

Epoch 11/1000:
Error: 0.11080236693482785

Epoch 12/1000:
Error: 0.09772373737121894

Epoch 13/1000:
Error: 0.0936286432537139

Epoch 14/1000:
Error: 0.09135425347010624

Epoch 15/1000:
Error: 0.0891223934426021

Epoch 16/1000:
Error: 0.08763342240431415

Epoch 17/1000:
Error: 0.08598828534255488

Epoch 18/1000:
Error: 0.08388228024763186

Epoch 19/1000:
Error: 0.08181312205949909

Epoch 20/1000:
Error: 0.07978081835127811

Epoch 21/1000:
Error: 0.07998199179338716

Epoch 22/1000:
Error: 0.07700583281705192

Epoch 23/1000:
Error: 0.07507249324161949

Epoch 24/1000:
Error: 0.07

In [61]:
pred = model.predict(X_test)
print(f'prediction: {pred}')
pred = np.around(pred)
print(f"Prediction rounded: {pred}")

prediction: [[1.23912536e-03 9.86391159e-01 5.95919673e-05]]
Prediction rounded: [[0. 1. 0.]]


In [62]:
accuracy_score(y_test, pred.flatten())

1.0

#### OR

In [63]:
y_train = np.array([0,1,1,1])

model = get_model()
model.fit(X_train, y_train, epochs=1000)

Epoch 1/1000:
Error: 0.3061785111565531

Epoch 2/1000:
Error: 0.2798274601582151

Epoch 3/1000:
Error: 0.23423429649807098

Epoch 4/1000:
Error: 0.1946154158393021

Epoch 5/1000:
Error: 0.16144785610047438

Epoch 6/1000:
Error: 0.13448324976055467

Epoch 7/1000:
Error: 0.12082079940474529

Epoch 8/1000:
Error: 0.11011533205610993

Epoch 9/1000:
Error: 0.10222180717810116

Epoch 10/1000:
Error: 0.09598693187620061

Epoch 11/1000:
Error: 0.09101282721672371

Epoch 12/1000:
Error: 0.08726319229372945

Epoch 13/1000:
Error: 0.08432908518249316

Epoch 14/1000:
Error: 0.08192551227077312

Epoch 15/1000:
Error: 0.07985641138283657

Epoch 16/1000:
Error: 0.07798854152342956

Epoch 17/1000:
Error: 0.07623260946897933

Epoch 18/1000:
Error: 0.07452987220171121

Epoch 19/1000:
Error: 0.07284272478256311

Epoch 20/1000:
Error: 0.07114702213472085

Epoch 21/1000:
Error: 0.06942440988350002

Epoch 22/1000:
Error: 0.06767613338127235

Epoch 23/1000:
Error: 0.06590021165407293

Epoch 24/1000:
Error: 0

In [64]:
pred = model.predict(X_test)
print(f'prediction: {pred}')
pred = np.around(pred)
print(f"Prediction rounded: {pred}")

prediction: [[0.99883923 0.99999993 0.09859692]]
Prediction rounded: [[1. 1. 0.]]


#### XOR

Dit keer met wat minder epochs en minder layers om verschilt te zien

In [65]:
y_train = np.array([0, 1,1,0])
model = get_model()

model.fit(X_train, y_train, epochs=150)

Epoch 1/150:
Error: 0.37534868309762226

Epoch 2/150:
Error: 0.48932821053128667

Epoch 3/150:
Error: 0.46233436194959976

Epoch 4/150:
Error: 0.4426462132208147

Epoch 5/150:
Error: 0.42540959017201885

Epoch 6/150:
Error: 0.41010820692204775

Epoch 7/150:
Error: 0.3964132538587668

Epoch 8/150:
Error: 0.3840516891245518

Epoch 9/150:
Error: 0.3727982965858805

Epoch 10/150:
Error: 0.36246878576185126

Epoch 11/150:
Error: 0.3529125400764485

Epoch 12/150:
Error: 0.34400607984160325

Epoch 13/150:
Error: 0.33564759827991314

Epoch 14/150:
Error: 0.32775256613971016

Epoch 15/150:
Error: 0.2806089564291694

Epoch 16/150:
Error: 0.3161233925183408

Epoch 17/150:
Error: 0.27283854815524045

Epoch 18/150:
Error: 0.30144123311950427

Epoch 19/150:
Error: 0.2656529722927678

Epoch 20/150:
Error: 0.2878276123332701

Epoch 21/150:
Error: 0.2583969152184806

Epoch 22/150:
Error: 0.27510449051193386

Epoch 23/150:
Error: 0.25188582736474713

Epoch 24/150:
Error: 0.26629681940512706

Epoch 25/15

In [66]:
pred = model.predict(X_test)
print(f'prediction: {pred}')
pred = np.around(pred)
print(f"Prediction rounded: {pred}")

prediction: [[0.89498898 0.05289843 0.05934472]]
Prediction rounded: [[1. 0. 0.]]


In [67]:
accuracy_score(y_test, pred.flatten())

0.3333333333333333

### Evaluatie

De Sequential model hierboven is een simpele neurale netwerk. Niet al te best natuurlijk, maar het is een begin. Het werkt wel degelijk. De softmax functie heb ik niet werkend gekregen. Daarom is alleen een regressie en binary classification voorbeeld laten zien.